In [1]:
import pandas as pd
import pymysql
from pyhive import hive
import pandas as pd
from pyspark import SparkContext,SQLContext
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.types import StringType
import datetime

In [2]:
class hiveParse(object):
    """
    该方法用于读取hive 数据，并转化成pandas dataframe
    """
    def __init__(self, host='', port=10000, user='', passwd='', dbName=''):
        self.host = host
        self.port = port
        self.user = user
        self.passwd = passwd
        self.dbName = dbName

    def conn_hive(self):
        self.connection = hive.Connection(host=self.host,port=self.port,username=self.user)

    def hive_connect(self,sql_select):
        cur = self.connection.cursor()
        cur.execute(sql_select)
        columns = [col[0] for col in cur.description]
        data = cur.fetchall()
        query_df = pd.DataFrame(data)
        query_df.columns = columns
        return query_df

class mysqldbParse(object):
    """
    该方法用于连接mysql，并执行相应操作
    """

    def __init__(self, host='', port=3306, user='', passwd='', dbName='', charset='utf8', connect_timeout=31536000):
        self.host = host
        self.port = port
        self.user = user
        self.passwd = passwd
        self.dbName = dbName
        self.charset = charset
        self.connect_timeout = connect_timeout

    def conn_mysql(self):
        self.conn = pymysql.connect(host=self.host,
                                    port=self.port,
                                    user=self.user,
                                    password=self.passwd,
                                    db=self.dbName,
                                    charset=self.charset,
                                    connect_timeout=self.connect_timeout,
                                    cursorclass=pymysql.cursors.DictCursor)

    def insert_sql(self, insertSql):
        with self.conn.cursor() as cursor:
            cursor.execute(insertSql)
            self.conn.commit()

    def select_Sql(self, selectSql):
        df = pd.read_sql(selectSql, self.conn)
        return df

    def close(self):
        self.conn.close()

In [3]:
def hive_df(sql):
    host = "58.59.18.61"
    port = 10000
    user = "hdfs"
    dbName = "ods"
    hive_parse = hiveParse(host,port,user,dbName)
    hive_parse.conn_hive()
    hive_pd = hive_parse.hive_connect(sql)
    return hive_pd

def mysql_df(selectSql):
    host = "58.59.11.86"
    port = 3306
    user = "tangdinghai"
    passwd = "Tangdinghai2019!"
    dbName = "reportpublic"
    mysql_parse = mysqldbParse(host,port,user,passwd,dbName)
    mysql_parse.conn_mysql()
    mysql_pd = mysql_parse.select_Sql(selectSql)
    mysql_parse.close()
    return mysql_pd



In [4]:
#### 有授信额度相关客户级和订单级汇总监控报表---日报

In [5]:
mysql_sql =  """

select
	score_flag as score_interval,
	count(distinct(user_id)) as shouxin_uid_%s,
    count(distinct(CREDIT_APPLY_ID)) as shouxin_applyid_%s

from
(
	select
		user_id,
        CREDIT_APPLY_ID,
        create_time,
		case
			when t.score >= 409 and t.score <= 560 then "409-560"
			when t.score >= 561 and t.score <= 583 then "561-583"
			when t.score >= 584 and t.score <= 597 then "584-597"
			when t.score >= 598 and t.score <= 608 then "598-608"
			when t.score >= 609 and t.score <= 618 then "609-618"
			when t.score >= 619 and t.score <= 628 then "619-628"
			when t.score >= 629 and t.score <= 638 then "629-638"
			when t.score >= 639 and t.score <= 649 then "639-649"
			when t.score >= 650 and t.score <= 664 then "650-664"
			when t.score >= 665 and t.score <= 752 then "665-752"
		end as score_flag
	from 
    
             (
                select 
                        b.USER_ID as user_id,
                        b.CREDIT_APPLY_ID as CREDIT_APPLY_ID,
                        b.APPLY_STATUS as APPLY_STATUS,
                        b.create_time as create_time,
                        a.SCORE as score
                from
                (
                    select 
                            USER_ID,
                            CREDIT_ID,
                            CREDIT_APPLY_ID,
                            CREDIT_AMOUNT,
                            SCORE,
                            PRODUCT_NO
                    from ods_credit_credit_info where product_no like 'PN00000053%%'
                    ) as a 

                    right join
                    (
                    select 
                            TENANT_ID,
                            USER_ID,
                            CREDIT_APPLY_ID,
                            APPLY_STATUS,
                            create_time
                    from ods_credit_credit_apply
                    where product_no like 'PN00000053%%'
                    ) as b
                    on a.USER_ID = b.USER_ID
                ) as t
                  where date(t.create_time) 
                  between %s  and %s  -- 今天减去一天
) as k group by k.score_flag


"""
# table_day = mysql_df(sql_day)

In [11]:
daily_shouxin_month = dairly_monitor_shouxin(mysql_sql,'month',1,2)
daily_shouxin_month

,score_interval,shouxin_uid_month,shouxin_applyid_month
0,None,1401,1402
1,584-597,3,3
2,598-608,15,15
3,609-618,21,21
4,619-628,22,22
5,629-638,26,26
6,639-649,10,10
7,650-664,26,26
8,665-752,18,18


In [6]:
def dairly_monitor_shouxin(sql,flag,daydiff1=0,daydiff2=0):
    
    if(flag == 'today'):
        time1 = """ADDDATE(curdate(),-1)"""
        time2 = """ADDDATE(curdate(),-1)"""
        flag = 'today'
        excute_sql = sql%(flag,flag,time1,time2)
#         print(excute_sql)
    if(flag == 'week'):
        time1 = """ADDDATE(curdate(),-%d)"""%daydiff1
        time2 = """ADDDATE(curdate(),-%d)"""%daydiff2
        excute_sql = sql%(flag,flag,time1,time2)
        
    if(flag == 'month'):
        time1 = """ADDDATE(ADDDATE(curdate(),INTERVAL -%d month),-%d)"""%(daydiff1,daydiff2)
        time2 = """ADDDATE(curdate(),-%d)"""%daydiff2
        excute_sql = sql%(flag,flag,time1,time2)
    #获取每个月的时间
    credit_apply = mysql_df(excute_sql)
    return credit_apply

In [7]:
daily_shouxin_day = dairly_monitor_shouxin(mysql_sql,'today',0,0)
daily_shouxin_week = dairly_monitor_shouxin(mysql_sql,'week',8,2)
daily_shouxin_month = dairly_monitor_shouxin(mysql_sql,'month',1,2)



base_dict = {'score_interval':['409-560','561-583','584-597','598-608','609-618','619-628','629-638','639-649','650-664','665-752','汇总'],\
             'toutiao_develop':['2.80%','8.09%','10.02%','10.57%','10.63%','11.82%','11.28%','11.31%','11.67%','11.81%','-']}
dairy_report = pd.DataFrame(base_dict)


shouxin_monitor_dairly = dairy_report.merge(daily_shouxin_month,on = 'score_interval',how='left')\
                                  .merge(daily_shouxin_week,on = 'score_interval',how='left')\
                                  .merge(daily_shouxin_day,on = 'score_interval',how='left').fillna(0)


col_keep = list(shouxin_monitor_dairly.columns)
col_keep.remove('score_interval')
col_keep.remove('toutiao_develop')

for col in col_keep:
    col_total = sum(shouxin_monitor_dairly[col])
    shouxin_monitor_dairly[col][10] = col_total
    shouxin_monitor_dairly[col+"_percent"] = shouxin_monitor_dairly[col].map(lambda x: x/(col_total+0.000001))
    shouxin_monitor_dairly[col+"_percent"][10] = col_total 

    
shouxin_monitor_dairly



/root/.pyenv/versions/3.5.1/envs/vpy3.5/lib/python3.5/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/root/.pyenv/versions/3.5.1/envs/vpy3.5/lib/python3.5/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,score_interval,toutiao_develop,shouxin_uid_month,shouxin_applyid_month,shouxin_uid_week,shouxin_applyid_week,shouxin_uid_today,shouxin_applyid_today,shouxin_uid_month_percent,shouxin_applyid_month_percent,shouxin_uid_week_percent,shouxin_applyid_week_percent,shouxin_uid_today_percent,shouxin_applyid_today_percent
0,409-560,2.80%,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0
1,561-583,8.09%,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0
2,584-597,10.02%,3.0,3.0,2.0,2.0,0.0,0.0,0.021277,0.021277,0.018349,0.018349,0.0,0.0
3,598-608,10.57%,15.0,15.0,14.0,14.0,0.0,0.0,0.106383,0.106383,0.128440,0.128440,0.0,0.0
4,609-618,10.63%,21.0,21.0,14.0,14.0,0.0,0.0,0.148936,0.148936,0.128440,0.128440,0.0,0.0
5,619-628,11.82%,22.0,22.0,17.0,17.0,0.0,0.0,0.156028,0.156028,0.155963,0.155963,0.0,0.0
6,629-638,11.28%,26.0,26.0,21.0,21.0,0.0,0.0,0.184397,0.184397,0.192661,0.192661,0.0,0.0
7,639-649,11.31%,10.0,10.0,10.0,10.0,0.0,0.0,0.070922,0.070922,0.091743,0.091743,0.0,0.0
8,650-664,11.67%,26.0,26.0,16.0,16.0,0.0,0.0,0.184397,0.184397,0.146789,0.146789,0.0,0.0
9,665-752,11.81%,18.0,18.0,15.0,15.0,0.0,0.0,0.127660,0.127660,0.137615,0.137615,0.0,0.0


In [8]:
spark=SparkSession \
        .builder \
        .config("spark.eventLog.enabled", "false") \
        .config("spark.executor.memory", "4g")\
        .config("spark.driver.memory", "8g")\
        .config("spark.cores.max", "10")\
        .config("spark.task.maxFailures", "1000")\
        .config("spark.default.parallelism", "500")\
        .config("spark.sql.shuffle.partitions",50)\
        .appName('renhang_etl') \
        .master('yarn')\
        .getOrCreate()




In [ ]:
spark_df = spark.createDataFrame(application_monitor).createOrReplaceTempView("application_monitor")

In [287]:
import datetime
now_time = (datetime.datetime.now()+datetime.timedelta(days=-1)).strftime('%Y-%m-%d')
now_time


sql =  """

insert overwrite table renhang_user_profile.apply_monitor_user_day partition(dt='%s') 
select 
    score_interval,
    toutiao_develop,
    apply_day_percent,
    apply_week_percent,
    apply_month_percent
from application_monitor

""" %now_time

spark.sql(sql)


DataFrame[]

In [288]:
###########################################
#### 有授信额度相关客户级和订单级汇总监控报表---月报

In [35]:
# my_sql =  """

# select
# 	score_flag as score_interval,
# 	count(distinct(user_id)) as apply_num_%s
# from
# (
# 	select
# 		user_id,
#         create_time,
# 		case
# 			when (t.score ) > 680 then ">680"
# 			when (t.score ) > 650 and (t.score ) <= 680 then "(650,680]"
# 			when (t.score ) > 630 and (t.score ) <= 650 then "(630,650]"
# 			when (t.score ) > 610 and (t.score ) <= 630 then "(610,630]"
# 			when (t.score ) > 595 and (t.score ) <= 610 then "(595,610]"
# 			when (t.score ) <= 595 then "<=595"
# 		end as score_flag
# 	from 
    
#              (
#                 select 
#                         b.USER_ID as user_id,
#                         b.APPLY_STATUS as APPLY_STATUS,
#                         b.create_time as create_time,
#                         a.SCORE as score
#                 from
#                 (
#                     select 
#                             USER_ID,
#                             CREDIT_ID,
#                             CREDIT_APPLY_ID,
#                             CREDIT_AMOUNT,
#                             SCORE,
#                             PRODUCT_NO
#                     from ods_credit_credit_info where product_no like 'PN00000053%%'
#                     ) as a 

#                     right join
#                     (
#                     select 
#                             TENANT_ID,
#                             USER_ID,
#                             CREDIT_APPLY_ID,
#                             APPLY_STATUS,
#                             create_time
#                     from ods_credit_credit_apply
#                     where product_no like 'PN00000053%%'
#                     ) as b
#                     on a.USER_ID = b.USER_ID
#                 ) as t
#                   where date(t.create_time) between 
#                     %s and %s

# ) as k group by k.score_flag

# """
# # credit_apply = mysql_df(sql)

In [68]:
def monthly_monitor_shouxin(sql,flag,monthdiff):
    
    if(flag == 'online'):
        time1 = """ADDDATE(DATE_ADD(STR_TO_DATE('2019-01-14',"%Y-%m-%d"),interval -day(STR_TO_DATE('2019-01-14',"%Y-%m-%d"))+1 day),  interval -0 month)"""
        time2 = """last_day(STR_TO_DATE('2019-01-14',"%Y-%m-%d"))"""
        flag = 'online'
        excute_sql = sql%(flag,flag,time1,time2)

    else:
        time1 = """ADDDATE(DATE_ADD(curdate(),interval -day(curdate())+1 day),  interval -%d month)"""%monthdiff
        time2 = """ADDDATE(last_day(curdate()),interval -%d month)"""%monthdiff
        date_sql = """select ADDDATE(last_day(curdate()),interval -%d month) as date"""%monthdiff
        date =  mysql_df(date_sql)['date'].astype('str')
        flag = date[0][0:7].replace('-','_')
        excute_sql = sql%(flag,flag,time1,time2)

    #获取每个月的时间
    credit_apply = mysql_df(excute_sql)
    return credit_apply

In [69]:
#上线月
shouxin_month_online = monthly_monitor_shouxin(mysql_sql,'online',-1)
#当月
shouxin_month_now = monthly_monitor_shouxin(mysql_sql,'not_online',0)
#上月
shouxin_month_last1 = monthly_monitor_shouxin(mysql_sql,'not_online',1)
#上两月
shouxin_month_last2 = monthly_monitor_shouxin(mysql_sql,'not_online',2)
#上3月
shouxin_month_last3 = monthly_monitor_shouxin(mysql_sql,'not_online',3)
#上4月
shouxin_month_last4 = monthly_monitor_shouxin(mysql_sql,'not_online',4)
#上5月
shouxin_month_last5 = monthly_monitor_shouxin(mysql_sql,'not_online',5)
#上6月
shouxin_month_last6 = monthly_monitor_shouxin(mysql_sql,'not_online',6)


In [70]:
shouxin_monitor_monthly = dairy_report.merge(shouxin_month_online,on = 'score_interval',how='left')\
                                  .merge(shouxin_month_last6,on = 'score_interval',how='left')\
                                  .merge(shouxin_month_last5,on = 'score_interval',how='left')\
                                  .merge(shouxin_month_last4,on = 'score_interval',how='left')\
                                  .merge(shouxin_month_last3,on = 'score_interval',how='left')\
                                  .merge(shouxin_month_last2,on = 'score_interval',how='left')\
                                  .merge(shouxin_month_last1,on = 'score_interval',how='left')\
                                  .merge(shouxin_month_now,on = 'score_interval',how='left').fillna(0)

col_keep = list(shouxin_monitor_monthly.columns)
col_keep.remove('score_interval')
col_keep.remove('toutiao_develop')

for col in col_keep:
    col_total = sum(shouxin_monitor_monthly[col])
    shouxin_monitor_monthly[col][6] = col_total
    shouxin_monitor_monthly[col+"_percent"] = shouxin_monitor_monthly[col].map(lambda x: x/(col_total+0.000001))
    shouxin_monitor_monthly[col+"_percent"][6] = col_total 

    
shouxin_monitor_monthly

/root/.pyenv/versions/3.5.1/envs/vpy3.5/lib/python3.5/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/root/.pyenv/versions/3.5.1/envs/vpy3.5/lib/python3.5/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,score_interval,toutiao_develop,shouxin_uid_online,shouxin_applyid_online,shouxin_uid_2018_07,shouxin_applyid_2018_07,shouxin_uid_2018_08,shouxin_applyid_2018_08,shouxin_uid_2018_09,shouxin_applyid_2018_09,...,shouxin_uid_2018_09_percent,shouxin_applyid_2018_09_percent,shouxin_uid_2018_10_percent,shouxin_applyid_2018_10_percent,shouxin_uid_2018_11_percent,shouxin_applyid_2018_11_percent,shouxin_uid_2018_12_percent,shouxin_applyid_2018_12_percent,shouxin_uid_2019_01_percent,shouxin_applyid_2019_01_percent
0,>680,24.89%,4.0,4.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.034783,0.034783
1,"(650,680]",20.10%,35.0,35.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.304348,0.304348
2,"(630,650]",17.00%,24.0,24.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.208696,0.208696
3,"(610,630]",14.87%,36.0,36.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.313043,0.313043
4,"(595,610]",14.29%,15.0,15.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.130435,0.130435
5,<=595,10.85%,1.0,1.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.008696,0.008696
6,汇总,21590,115.0,115.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,115.000000,115.000000


In [71]:
#### 申请监控订单级和客户级日报 --- from hive

In [72]:
hive_sql = """


select
	score_interval,
	count(distinct(user_id)) as apply_uid_%s,
    count(distinct(session_id)) as apply_sessionId_%s
from
(
	select
		user_id,
        session_id,
		case
			when cast(t.out_credit_score as int) > 680 then ">680"
			when cast(t.out_credit_score as int) > 650 and cast(t.out_credit_score as int) <= 680 then "(650,680]"
			when cast(t.out_credit_score as int) > 630 and cast(t.out_credit_score as int) <= 650 then "(630,650]"
			when cast(t.out_credit_score as int) > 610 and cast(t.out_credit_score as int) <= 630 then "(610,630]"
			when cast(t.out_credit_score as int) > 595 and cast(t.out_credit_score as int) <= 610 then "(595,610]"
			when cast(t.out_credit_score as int) <= 595 then "<=595"
		end as score_interval
	from 
			(
				SELECT
     				a.session_id as session_id,
     				a.user_id as user_id,
     				a.t02_apply_time as apply_time,
     				b.out_credit_score as out_credit_score
				FROM 
				(
					SELECT 
						session_id,
       					user_id,
       					t02_apply_time
					FROM ods.ods_toutiao_shouxin_input 
				) AS a

			left JOIN
  			(
  				SELECT 
  					session_id,
  					user_id,
          			out_credit_score
   				FROM ods.ods_toutiao_shouxin_output
  			) AS b ON a.session_id = b.session_id
		) as t  where from_unixtime(unix_timestamp(substr(t.apply_time,0,8),'yyyymmdd'),'yyyy-mm-dd') 
            between %s and %s  
) as a group by a.score_interval

"""
# toutiao = hive_df(hive_sql)

In [73]:
def dairly_monitor_apply(sql,flag,daydiff1=0,daydiff2=0):
    
    if(flag == 'today'):
        time1 = """date_add(CURRENT_DATE,-1)"""
        time2 = """date_add(CURRENT_DATE,-1)"""
        flag = 'today'
        excute_sql = sql%(flag,flag,time1,time2)
#         print(excute_sql)
    if(flag == 'week'):
        time1 = """date_add(CURRENT_DATE,-%d)"""%daydiff1
        time2 = """date_add(CURRENT_DATE,-%d)"""%daydiff2
        excute_sql = sql%(flag,flag,time1,time2)
        
    if(flag == 'month'):
        time1 = """date_add(add_months(CURRENT_DATE, -%d ),-%d)"""%(daydiff1,daydiff2)
        time2 = """date_add(CURRENT_DATE,-%d)"""%daydiff2
        excute_sql = sql%(flag,flag,time1,time2)
    #获取每个月的时间
    credit_apply = hive_df(excute_sql)
    return credit_apply

In [74]:
dairly_apply_day = dairly_monitor_apply(hive_sql,'today',0,0)
dairly_apply_week = dairly_monitor_apply(hive_sql,'week',8,2)
dairly_apply_month = dairly_monitor_apply(hive_sql,'month',1,2)


# 固定的两列
base_dict = {'score_interval':['>680','(650,680]','(630,650]','(610,630]','(595,610]','<=595','汇总'],\
             'toutiao_develop':['24.89%','20.10%','17.00%','14.87%','14.29%','10.85%','21590']}
dairy_report = pd.DataFrame(base_dict)


apply_monitor_dairly = dairy_report.merge(dairly_apply_month,on = 'score_interval',how='left')\
                                  .merge(dairly_apply_week,on = 'score_interval',how='left')\
                                  .merge(dairly_apply_day,on = 'score_interval',how='left').fillna(0)

col_keep = list(apply_monitor_dairly.columns)
col_keep.remove('score_interval')
col_keep.remove('toutiao_develop')

for col in col_keep:
    col_total = sum(apply_monitor_dairly[col])
    apply_monitor_dairly[col][6] = col_total
    apply_monitor_dairly[col+"_percent"] = apply_monitor_dairly[col].map(lambda x: x/(col_total+0.000001))
    apply_monitor_dairly[col+"_percent"][6] = col_total 

    
apply_monitor_dairly

/root/.pyenv/versions/3.5.1/envs/vpy3.5/lib/python3.5/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/root/.pyenv/versions/3.5.1/envs/vpy3.5/lib/python3.5/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,score_interval,toutiao_develop,apply_uid_month,apply_sessionid_month,apply_uid_week,apply_sessionid_week,apply_uid_today,apply_sessionid_today,apply_uid_month_percent,apply_sessionid_month_percent,apply_uid_week_percent,apply_sessionid_week_percent,apply_uid_today_percent,apply_sessionid_today_percent
0,>680,24.89%,8.0,8.0,7.0,7.0,1.0,1.0,0.010088,0.010076,0.009655,0.009642,0.006173,0.006173
1,"(650,680]",20.10%,53.0,53.0,46.0,46.0,3.0,3.0,0.066835,0.066751,0.063448,0.063361,0.018519,0.018519
2,"(630,650]",17.00%,60.0,60.0,57.0,57.0,6.0,6.0,0.075662,0.075567,0.078621,0.078512,0.037037,0.037037
3,"(610,630]",14.87%,110.0,110.0,99.0,99.0,14.0,14.0,0.138714,0.138539,0.136552,0.136364,0.086420,0.086420
4,"(595,610]",14.29%,113.0,113.0,105.0,105.0,26.0,26.0,0.142497,0.142317,0.144828,0.144628,0.160494,0.160494
5,<=595,10.85%,449.0,450.0,411.0,412.0,112.0,112.0,0.566204,0.566751,0.566897,0.567493,0.691358,0.691358
6,汇总,21590,793.0,794.0,725.0,726.0,162.0,162.0,793.000000,794.000000,725.000000,726.000000,162.000000,162.000000


In [75]:
#### 申请监控月报 --- from hive

In [80]:
hive_sql = """


select
	score_interval,
	count(distinct(user_id)) as apply_uid_%s,
    count(distinct(session_id)) as apply_sessionId_%s

from
(
	select
		user_id,
        session_id,
		case
			when cast(t.out_credit_score as int) > 680 then ">680"
			when cast(t.out_credit_score as int) > 650 and cast(t.out_credit_score as int) <= 680 then "(650,680]"
			when cast(t.out_credit_score as int) > 630 and cast(t.out_credit_score as int) <= 650 then "(630,650]"
			when cast(t.out_credit_score as int) > 610 and cast(t.out_credit_score as int) <= 630 then "(610,630]"
			when cast(t.out_credit_score as int) > 595 and cast(t.out_credit_score as int) <= 610 then "(595,610]"
			when cast(t.out_credit_score as int) <= 595 then "<=595"
		end as score_interval
	from 
			(
				SELECT
     				a.session_id as session_id,
     				a.user_id as user_id,
     				a.t02_apply_time as apply_time,
     				b.out_credit_score as out_credit_score
				FROM 
				(
					SELECT 
						session_id,
       					user_id,
       					t02_apply_time
					FROM ods.ods_toutiao_shouxin_input 
				) AS a

			left JOIN
  			(
  				SELECT 
  					session_id,
  					user_id,
          			out_credit_score
   				FROM ods.ods_toutiao_shouxin_output
  			) AS b ON a.session_id = b.session_id
		) as t  where from_unixtime(unix_timestamp(substr(t.apply_time,0,8),'yyyymmdd'),'yyyy-mm-dd') 
            between %s and %s  
) as a group by a.score_interval

"""
# toutiao = hive_df(hive_sql)

In [81]:
def monthly_monitor_apply(sql,flag,monthdiff):
    
    if(flag == 'online'):
        time1 = """date_add(last_day(add_months('2019-01-14', -1 )),1)"""
        time2 = """last_day('2019-01-14')"""
        flag = 'online'
        excute_sql = sql%(flag,flag,time1,time2)
#         print(excute_sql)
    else:
        time1 = """date_add(last_day(add_months(CURRENT_DATE, -%d )),1)"""%(monthdiff+1)    #当月第一天：上月最后一天+1天
        time2 = """last_day(add_months(CURRENT_DATE,-%d))"""%monthdiff                  #当月最后一天
        
        date_sql = """select last_day(add_months(CURRENT_DATE,-%d)) as dt"""%monthdiff
        date =  hive_df(date_sql)['dt'].astype('str')
        flag = date[0][0:7].replace('-','_')
        excute_sql = sql%(flag,flag,time1,time2)
#         print(excute_sql)

    #获取每个月的时间
    credit_apply = spark.sql(excute_sql).toPandas()
    return credit_apply

In [82]:
#上线月
apply_month_online = monthly_monitor_apply(hive_sql,'online',-1)
#当月
apply_month_now = monthly_monitor_apply(hive_sql,'not_online',0)
#上月
apply_month_last1 = monthly_monitor_apply(hive_sql,'not_online',1)
#上两月
apply_month_last2 = monthly_monitor_apply(hive_sql,'not_online',2)
#上3月
apply_month_last3 = monthly_monitor_apply(hive_sql,'not_online',3)
#上4月
apply_month_last4 = monthly_monitor_apply(hive_sql,'not_online',4)
#上5月
apply_month_last5 = monthly_monitor_apply(hive_sql,'not_online',5)
#上6月
apply_month_last6 = monthly_monitor_apply(hive_sql,'not_online',6)


In [83]:
base_dict = {'score_interval':['>680','(650,680]','(630,650]','(610,630]','(595,610]','<=595','汇总'],\
             'toutiao_develop':['24.89%','20.10%','17.00%','14.87%','14.29%','10.85%','21590']}
dairy_report = pd.DataFrame(base_dict)

apply_monitor_monthly = dairy_report.merge(apply_month_online,on = 'score_interval',how='left')\
                                  .merge(apply_month_last6,on = 'score_interval',how='left')\
                                  .merge(apply_month_last5,on = 'score_interval',how='left')\
                                  .merge(apply_month_last4,on = 'score_interval',how='left')\
                                  .merge(apply_month_last3,on = 'score_interval',how='left')\
                                  .merge(apply_month_last2,on = 'score_interval',how='left')\
                                  .merge(apply_month_last1,on = 'score_interval',how='left')\
                                  .merge(apply_month_now,on = 'score_interval',how='left').fillna(0)

col_keep = list(apply_monitor_monthly.columns)
col_keep.remove('score_interval')
col_keep.remove('toutiao_develop')

for col in col_keep:
    col_total = sum(apply_monitor_monthly[col])
    apply_monitor_monthly[col][6] = col_total
    apply_monitor_monthly[col+"_percent"] = apply_monitor_monthly[col].map(lambda x: x/(col_total+0.000001))
    apply_monitor_monthly[col+"_percent"][6] = col_total 

    
apply_monitor_monthly

/root/.pyenv/versions/3.5.1/envs/vpy3.5/lib/python3.5/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/root/.pyenv/versions/3.5.1/envs/vpy3.5/lib/python3.5/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,score_interval,toutiao_develop,apply_uid_online,apply_sessionId_online,apply_uid_2018_07,apply_sessionId_2018_07,apply_uid_2018_08,apply_sessionId_2018_08,apply_uid_2018_09,apply_sessionId_2018_09,...,apply_uid_2018_09_percent,apply_sessionId_2018_09_percent,apply_uid_2018_10_percent,apply_sessionId_2018_10_percent,apply_uid_2018_11_percent,apply_sessionId_2018_11_percent,apply_uid_2018_12_percent,apply_sessionId_2018_12_percent,apply_uid_2019_01_percent,apply_sessionId_2019_01_percent
0,>680,24.89%,10.0,10.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.009930,0.009921
1,"(650,680]",20.10%,56.0,56.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.055611,0.055556
2,"(630,650]",17.00%,68.0,68.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.067527,0.067460
3,"(610,630]",14.87%,129.0,129.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.128103,0.127976
4,"(595,610]",14.29%,144.0,144.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.142999,0.142857
5,<=595,10.85%,600.0,601.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.595829,0.596230
6,汇总,21590,1007.0,1008.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1007.000000,1008.000000


In [56]:
####上述已经完成订单级汇总和客户级汇总
#### 申请表通过率

In [58]:
#### 申请表通过率

In [115]:
accept_rate_dairly = shouxin_monitor_dairly[['score_interval']]
for i in range(len(shouxin_monitor_dairly.columns)):
    if(i>=2 and i <=7):
        shouxin_col = shouxin_monitor_dairly.columns[i]
        apply_col = apply_monitor_dairly.columns[i]
#         print(shouxin_col,apply_col)

        accept_col = shouxin_col.split("_")[1]+"_"+shouxin_col.split("_")[2]
        accept_rate_dairly[accept_col+"_rate"] = shouxin_monitor_dairly[shouxin_col]/(apply_monitor_dairly[apply_col])
        accept_rate_dairly[accept_col+"_rate"][6] = apply_monitor_dairly[apply_col][6]

accept_rate_dairly

/root/.pyenv/versions/3.5.1/envs/vpy3.5/lib/python3.5/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/root/.pyenv/versions/3.5.1/envs/vpy3.5/lib/python3.5/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


,score_interval,uid_month_rate,applyid_month_rate,uid_week_rate,applyid_week_rate,uid_today_rate,applyid_today_rate
0,>680,0.375000,0.375000,0.285714,0.285714,1.000000,1.000000
1,"(650,680]",0.641509,0.641509,0.630435,0.630435,0.333333,0.333333
2,"(630,650]",0.350000,0.350000,0.368421,0.368421,0.500000,0.500000
3,"(610,630]",0.290909,0.290909,0.313131,0.313131,0.285714,0.285714
4,"(595,610]",0.106195,0.106195,0.114286,0.114286,0.115385,0.115385
5,<=595,0.002227,0.002222,0.000000,0.000000,0.000000,0.000000
6,汇总,793.000000,794.000000,725.000000,726.000000,162.000000,162.000000


In [116]:
shouxin_monitor_monthly

,score_interval,toutiao_develop,shouxin_uid_online,shouxin_applyid_online,shouxin_uid_2018_07,shouxin_applyid_2018_07,shouxin_uid_2018_08,shouxin_applyid_2018_08,shouxin_uid_2018_09,shouxin_applyid_2018_09,...,shouxin_uid_2018_09_percent,shouxin_applyid_2018_09_percent,shouxin_uid_2018_10_percent,shouxin_applyid_2018_10_percent,shouxin_uid_2018_11_percent,shouxin_applyid_2018_11_percent,shouxin_uid_2018_12_percent,shouxin_applyid_2018_12_percent,shouxin_uid_2019_01_percent,shouxin_applyid_2019_01_percent
0,>680,24.89%,4.0,4.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.034783,0.034783
1,"(650,680]",20.10%,35.0,35.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.304348,0.304348
2,"(630,650]",17.00%,24.0,24.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.208696,0.208696
3,"(610,630]",14.87%,36.0,36.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.313043,0.313043
4,"(595,610]",14.29%,15.0,15.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.130435,0.130435
5,<=595,10.85%,1.0,1.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.008696,0.008696
6,汇总,21590,115.0,115.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,115.000000,115.000000


In [117]:
apply_monitor_monthly

,score_interval,toutiao_develop,apply_uid_online,apply_sessionId_online,apply_uid_2018_07,apply_sessionId_2018_07,apply_uid_2018_08,apply_sessionId_2018_08,apply_uid_2018_09,apply_sessionId_2018_09,...,apply_uid_2018_09_percent,apply_sessionId_2018_09_percent,apply_uid_2018_10_percent,apply_sessionId_2018_10_percent,apply_uid_2018_11_percent,apply_sessionId_2018_11_percent,apply_uid_2018_12_percent,apply_sessionId_2018_12_percent,apply_uid_2019_01_percent,apply_sessionId_2019_01_percent
0,>680,24.89%,10.0,10.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.009930,0.009921
1,"(650,680]",20.10%,56.0,56.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.055611,0.055556
2,"(630,650]",17.00%,68.0,68.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.067527,0.067460
3,"(610,630]",14.87%,129.0,129.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.128103,0.127976
4,"(595,610]",14.29%,144.0,144.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.142999,0.142857
5,<=595,10.85%,600.0,601.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.595829,0.596230
6,汇总,21590,1007.0,1008.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1007.000000,1008.000000


In [123]:
accept_rate_monthly = shouxin_monitor_monthly[['score_interval']]
for i in range(len(shouxin_monitor_monthly.columns)):
    if(i>=2 and i <=16):
        shouxin_col = shouxin_monitor_monthly.columns[i]
        apply_col = apply_monitor_monthly.columns[i]
#       print(shouxin_col,apply_col)

        accept_col = shouxin_col[8:]
        accept_rate_monthly[accept_col+"_rate"] = shouxin_monitor_monthly[shouxin_col]/(apply_monitor_monthly[apply_col])
        accept_rate_monthly[accept_col+"_rate"][6] = apply_monitor_monthly[apply_col][6]

accept_rate_monthly = accept_rate_monthly.fillna("0")
accept_rate_monthly


/root/.pyenv/versions/3.5.1/envs/vpy3.5/lib/python3.5/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/root/.pyenv/versions/3.5.1/envs/vpy3.5/lib/python3.5/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/root/.pyenv/versions/3.5.1/envs/vpy3.5/lib/python3.5/site-packages/IPython/core/interactiveshell.py:3267: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://p

,score_interval,uid_online_rate,applyid_online_rate,uid_2018_07_rate,applyid_2018_07_rate,uid_2018_08_rate,applyid_2018_08_rate,uid_2018_09_rate,applyid_2018_09_rate,uid_2018_10_rate,applyid_2018_10_rate,uid_2018_11_rate,applyid_2018_11_rate,uid_2018_12_rate,applyid_2018_12_rate,uid_2019_01_rate
0,>680,0.400000,0.400000,0,0,0,0,0,0,0,0,0,0,0,0,0.400000
1,"(650,680]",0.625000,0.625000,0,0,0,0,0,0,0,0,0,0,0,0,0.625000
2,"(630,650]",0.352941,0.352941,0,0,0,0,0,0,0,0,0,0,0,0,0.352941
3,"(610,630]",0.279070,0.279070,0,0,0,0,0,0,0,0,0,0,0,0,0.279070
4,"(595,610]",0.104167,0.104167,0,0,0,0,0,0,0,0,0,0,0,0,0.104167
5,<=595,0.001667,0.001664,0,0,0,0,0,0,0,0,0,0,0,0,0.001667
6,汇总,1007.000000,1008.000000,0,0,0,0,0,0,0,0,0,0,0,0,1007.000000


In [124]:
##### 头条征信评分相关报告